## Outline

[University of Liverpool - Ion Switching](https://www.kaggle.com/c/liverpool-ion-switching/overview)  
In this competition, We will be predicting the number of open_channels present, based on electrophysiological signal data.

IMPORTANT: While the time series appears continuous, the data is from discrete batches of 50 seconds long 10 kHz samples (500,000 rows per batch). In other words, the data from 0.0001 - 50.0000 is a different batch than 50.0001 - 100.0000, and thus discontinuous between 50.0000 and 50.0001.

### Data  
- Train data : 0.0001sec~ 500.0000sec   
- Testdata   : 500.0001sec~ 700.0000sec  

### Modeling
- LightGBM

### Metric  
- RMSE

### CV
- 5 stratified KFold

### Features
- gradients
- ema
- rolling_features
 - diff,lag,lead,max,min,mean,std,etc
- static_batch_features
 - norm,mean_abs_chg,max-mean,abs_avg,etc


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ion-shifted-rfc-proba/Y_train_proba.npy
/kaggle/input/ion-shifted-rfc-proba/Y_test_proba.npy
/kaggle/input/data-without-drift/test_clean.csv
/kaggle/input/data-without-drift/train_clean.csv
/kaggle/input/single-model-lgbm-kalman-filter/__results__.html
/kaggle/input/single-model-lgbm-kalman-filter/__output__.json
/kaggle/input/single-model-lgbm-kalman-filter/custom.css
/kaggle/input/single-model-lgbm-kalman-filter/__notebook__.ipynb
/kaggle/input/single-model-lgbm-kalman-filter/submission.csv
/kaggle/input/liverpool-ion-switching/train.csv
/kaggle/input/liverpool-ion-switching/test.csv
/kaggle/input/liverpool-ion-switching/sample_submission.csv


In [2]:
import math
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import lightgbm as lgb
from sklearn.model_selection import GroupKFold, StratifiedKFold, train_test_split
from sklearn import metrics
from tqdm import tqdm
from scipy import signal
import time
%matplotlib inline

## Read and cleaning data

In [3]:
# read data
def read_data():
    print('Reading training, testing and submission data...')
    train = pd.read_csv('/kaggle/input/data-without-drift/train_clean.csv')
    test = pd.read_csv('/kaggle/input/data-without-drift/test_clean.csv')
    submission = pd.read_csv('/kaggle/input/liverpool-ion-switching/sample_submission.csv', dtype={'time':str})
    train.iloc[478587:478588, [1]] =  -2.7098 # batch=1 out_remove,signal_median_b1
    train.iloc[478609:478610, [1]] = -2.7098 #
    print('Train set has {} rows and {} columns'.format(train.shape[0], train.shape[1]))
    print('Test set has {} rows and {} columns'.format(test.shape[0], test.shape[1]))
    return train, test, submission

#get batch
def get_batch(train, test):
    # concatenate data
    batch = 50 # 50sec/batch
    total_batches = 14
    train['set'] = 'train'
    test['set'] = 'test'
    data = pd.concat([train, test], sort=True)
    for i in range(int(total_batches)):
        data.loc[(data['time'] > i * batch) & (data['time'] <= (i + 1) * batch), 'batch'] = i + 1
    train = data[data['set'] == 'train']
    test = data[data['set'] == 'test']
    train.drop(['set'], inplace = True, axis = 1)
    test.drop(['set'], inplace = True, axis = 1)
    del data
    return train, test

# batch = 50 # 50sec/batch
# total_batches = 10
# for i in range(int(total_batches)):
#     train.loc[(train['time'] > i * batch) & (train['time'] <= (i + 1) * batch),'batch'] = i + 1

# メモリ削減
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        if col!='open_channels': #'open_channels'の数値を変換する
            col_type = df[col].dtypes
            if col_type in numerics: #'open_channels'の型がnumericsにあるか
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int': #'open_channels'の型頭3文字か
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max: #最小値がー128より大かつ、最大値が127より小さい時はint８
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)  
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
# #plot

# def plot_1(df,x1,y1):
#     plt.subplots(figsize=(19,5))
#     plt.plot(df[x1], df[y1], label=y1)
#     plt.legend(loc='best')
#     plt.grid()
    

# def plot_2(df,x1,y1,y2):
#     plt.subplots(figsize=(19,5))
#     plt.plot(df[x1], df[y1], label=y1)
#     plt.plot(df[x1], df[y2], label=y2,c='r', alpha=.5)
#     plt.legend(loc='best')
#     plt.grid()
    
# def plot_3(df,x1,y1,y2,y3):
#     plt.subplots(figsize=(19,5))
#     plt.plot(df[x1], df[y1], label=y1)
#     plt.plot(df[x1], df[y2], label=y2,c='r', alpha=.5)
#     plt.plot(df[x1], df[y3], label=y3,c='g', alpha=.5)
#     plt.legend(loc='best')
#     plt.grid()
    
# plot_2(train,'time', 'signal', 'open_channels')
# plot_1(test, 'time', 'signal')

# Feature engineering

In [5]:
# batch_1 = train[train['batch'] ==1]
# batch_1

In [6]:
#勾配
def calc_gradients(s, n_grads = 4):
    grads = pd.DataFrame()
    
    g = s.values
    for i in range(n_grads):
        g = np.gradient(g)
        grads['grad_' + str(i+1)] = g
        
    return grads

#指数移動関数
def calc_ewm(s, windows=[10, 50, 100, 500, 1000]): #10, 50, 100, 500, 1000 #指数移動関数
    ewm = pd.DataFrame()
    for w in windows:
        ewm['ewm_mean_' + str(w)] = s.ewm(span=w, min_periods=1).mean()
        ewm['ewm_std_' + str(w)] = s.ewm(span=w, min_periods=1).std()
        
    # add zeros when na values (std)
    ewm = ewm.fillna(value=0)
        
    return ewm


def add_features(s):
    gradients = calc_gradients(s)
    ewm = calc_ewm(s)
    
    return pd.concat([s, gradients, ewm], axis=1) #[s, gradients, low_pass, high_pass, ewm]

# normalize
def divide_and_add_features(s, signal_size=500000):
    s = s / 15.0
    
    ls = []
    for i in tqdm(range(int(s.shape[0]/signal_size))):
        sig = s[i*signal_size:(i+1)*signal_size].copy().reset_index(drop=True)
        sig_featured = add_features(sig)
        ls.append(sig_featured)
    
    return pd.concat(ls, axis=0)

In [7]:
def rolling_features(train, test):
    pre_train = train.copy()
    pre_test = test.copy()
    
    for df in [pre_train, pre_test]:
        df['diff_1'] = df.groupby('batch')['signal'].diff() #add
        df['diff_2'] = df.groupby('batch')['signal'].diff().diff() #add
        
        for i in range(1,7):
            df['lag_t' + str(i)] = df.groupby('batch')['signal'].transform(lambda x: x.shift(i)) #遅れシフト
            df['lead_t' + str(i)] = df.groupby('batch')['signal'].transform(lambda x: x.shift(-i))
        
        for window in [10,100,1000]:
                #roll backwards#シフト関数(-1遅れ)+窓関数
                df['signalmean_t' + str(window)] = df.groupby(['batch'])['signal'].transform(lambda x: x.shift(1).rolling(window).mean())
                df['signalstd_t' + str(window)] = df.groupby(['batch'])['signal'].transform(lambda x: x.shift(1).rolling(window).std())
                df['signalvar_t' + str(window)] = df.groupby(['batch'])['signal'].transform(lambda x: x.shift(1).rolling(window).var())
                df['signalmin_t' + str(window)] = df.groupby(['batch'])['signal'].transform(lambda x: x.shift(1).rolling(window).min())
                df['signalmax_t' + str(window)] = df.groupby(['batch'])['signal'].transform(lambda x: x.shift(1).rolling(window).max())
                #最大値の切り捨てと最小値の切り上げの差
                min_max = (df['signal'] - df['signalmin_t' + str(window)]) / (df['signalmax_t' + str(window)] - df['signalmin_t' + str(window)])
                df['norm_t' + str(window)] = min_max * (np.floor(df['signalmax_t' + str(window)]) - np.ceil(df['signalmin_t' + str(window)]))
                df['max-mean' +str(window)]  = abs( df['signalmax_t' + str(window)] - df['signalmean_t' + str(window)]) #追加                
                # roll forward
                # シフト関数(1進み)+窓関数
                df['signalmean_t' + str(window) + '_lead'] = df.groupby(['batch'])['signal'].transform(lambda x: x.shift(- window - 1).rolling(window).mean())
                df['signalstd_t' + str(window) + '_lead'] = df.groupby(['batch'])['signal'].transform(lambda x: x.shift(- window - 1).rolling(window).std())
                df['signalvar_t' + str(window) + '_lead'] = df.groupby(['batch'])['signal'].transform(lambda x: x.shift(- window - 1).rolling(window).var())
                df['signalmin_t' + str(window) + '_lead'] = df.groupby(['batch'])['signal'].transform(lambda x: x.shift(- window - 1).rolling(window).min())
                df['signalmax_t' + str(window) + '_lead'] = df.groupby(['batch'])['signal'].transform(lambda x: x.shift(- window - 1).rolling(window).max())   
                min_max = (df['signal'] - df['signalmin_t' + str(window) + '_lead']) / (df['signalmax_t' + str(window) + '_lead'] - df['signalmin_t' + str(window) + '_lead'])
                df['norm_t' + str(window) + '_lead'] = min_max * (np.floor(df['signalmax_t' + str(window) + '_lead']) - np.ceil(df['signalmin_t' + str(window) + '_lead']))
                df['max-mean' +str(window) + '_lead']  = abs( df['signalmax_t' + str(window) + '_lead']  - df['signalmin_t' + str(window) + '_lead']) #追加
            
                for percent in [0.01, 0.05, 0.090, 0.095]:
                    df['p_' + str(percent) + str(window)] = df.groupby(['batch'])['signal'].transform(lambda x: x.shift(1).rolling(window).quantile(0.9)) #パーセンタイル
                    df['p_' + str(percent) + '-mean' +str(window)]  = abs( df['p_' + str(percent) + str(window)] - df['signalmean_t' + str(window)])
    del train, test, min_max
    return pre_train, pre_test

In [8]:
def static_batch_features(df, n): #n=50000
    
    df = df.copy()
    df.drop('batch', inplace = True, axis = 1)
    df = df.sort_values(by=['time']).reset_index(drop=True)
    df.index = ((df.time * 10000) - 1).values
    df['batch_' + str(n)] = df.index // n
    df['batch_index_' + str(n)] = df.index  - (df['batch_' + str(n)] * n)
    df['batch_slices_' + str(n)] = df['batch_index_' + str(n)]  // (n / 10)
    df['batch_slices2_' + str(n)] = df.apply(lambda r: '_'.join([str(r['batch_' + str(n)]).zfill(3), str(r['batch_slices_' + str(n)]).zfill(3)]), axis=1)

    for c in ['batch_' + str(n), 'batch_slices2_' + str(n)]:
        d = {}
        # -----------------------------------------------
        d['mean' + c] = df.groupby([c])['signal'].mean()
        d['median' + c] = df.groupby([c])['signal'].median()
        d['max' + c] = df.groupby([c])['signal'].max()
        d['min' + c] = df.groupby([c])['signal'].min()
        #d['std' + c] = df.groupby([c])['signal'].std() #remove
        d['p10' + c] = df.groupby([c])['signal'].apply(lambda x: np.percentile(x, 10))
        d['p25' + c] = df.groupby([c])['signal'].apply(lambda x: np.percentile(x, 25))
        d['p75' + c] = df.groupby([c])['signal'].apply(lambda x: np.percentile(x, 75))
        d['p90' + c] = df.groupby([c])['signal'].apply(lambda x: np.percentile(x, 90))
        #d['skew' + c] = df.groupby([c])['signal'].apply(lambda x: pd.Series(x).skew()) #remove
        #d['kurtosis' + c] = df.groupby([c])['signal'].apply(lambda x: pd.Series(x).kurtosis()) #remove
        min_max = (d['mean' + c] - d['min' + c]) / (d['max' + c] - d['min' + c])
        d['norm' + c] = min_max * (np.floor(d['max' + c]) - np.ceil(d['min' + c]))
        d['mean_abs_chg' + c] = df.groupby([c])['signal'].apply(lambda x: np.mean(np.abs(np.diff(x))))
        d['abs_max' + c] = df.groupby([c])['signal'].apply(lambda x: np.max(np.abs(x)))
        d['abs_min' + c] = df.groupby([c])['signal'].apply(lambda x: np.min(np.abs(x)))
        d['range' + c] = d['max' + c] - d['min' + c]
        d['maxtomin' + c] = d['max' + c] / d['min' + c]
        d['abs_avg' + c] = (d['abs_min' + c] + d['abs_max' + c]) / 2
        d['max-mean' +c] = abs(d['max' + c] - d['mean' + c]) #add
        d['p90-mean' + c] = abs(d['p90' + c] -d['mean' + c]) #add
        # -----------------------------------------------
        for v in d:
            df[v] = df[c].map(d[v].to_dict())

    for c in [c1 for c1 in df.columns if c1 not in ['time', 'signal', 'open_channels', 'batch', 'batch_' + str(n), 
                                                    'batch_index_' + str(n), 'batch_slices_' + str(n), 
                                                    'batch_slices2_' + str(n)]]:
        df[c + '_msignal'] = df[c] - df['signal']
        
    df.reset_index(drop = True, inplace = True)
        
    return df

In [9]:
#proba
def proba(train, test):
    pre_train = train.copy()
    pre_test = test.copy()
    Y_train_proba = np.load("/kaggle/input/ion-shifted-rfc-proba/Y_train_proba.npy")
    Y_test_proba = np.load("/kaggle/input/ion-shifted-rfc-proba/Y_test_proba.npy")
    
    for i in range(11):
        pre_train[f"proba_{i}"] = Y_train_proba[:, i]
        pre_test[f"proba_{i}"] = Y_test_proba[:, i]
    del train, test
    return pre_train, pre_test

In [ ]:
pre_train

## Model
lgbm with 5 stratified KFold

In [10]:
def run_lgb(pre_train, pre_test, features, params):
    
    kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
    target = 'open_channels'
    oof_pred = np.zeros(len(pre_train))
    y_pred = np.zeros(len(pre_test))
     
    for fold, (tr_ind, val_ind) in enumerate(kf.split(pre_train, pre_train[target])):
        x_train, x_val = pre_train[features].iloc[tr_ind], pre_train[features].iloc[val_ind]
        y_train, y_val = pre_train[target][tr_ind], pre_train[target][val_ind]
        train_set = lgb.Dataset(x_train, y_train)
        val_set = lgb.Dataset(x_val, y_val)
        
        model = lgb.train(params, train_set, num_boost_round = 10000, early_stopping_rounds = 50, 
                         valid_sets = [train_set, val_set], verbose_eval = 100)
        #early_stopping_rounds=50, eta=0,1なので10/0.1=100が目安
        oof_pred[val_ind] = model.predict(x_val)
        
        y_pred += model.predict(pre_test[features]) / kf.n_splits
        
    rmse_score = np.sqrt(metrics.mean_squared_error(pre_train[target], oof_pred))
    # 予測値を0~10の範囲内に
    oof_pred = np.round(np.clip(oof_pred, 0, 10)).astype(int)
    round_y_pred = np.round(np.clip(y_pred, 0, 10)).astype(int)
    f1 = metrics.f1_score(pre_train[target], oof_pred, average = 'macro')
    
    print(f'Our oof rmse score is {rmse_score}')
    print(f'Our oof macro f1 score is {f1}')
    return round_y_pred

In [12]:
# feature engineer part 1 (signal processing features)
train, test, submission = read_data()
train

Reading training, testing and submission data...
Train set has 5000000 rows and 3 columns
Test set has 2000000 rows and 2 columns


,time,signal,open_channels
0,0.0001,-2.760000,0
1,0.0002,-2.855700,0
2,0.0003,-2.407400,0
3,0.0004,-3.140400,0
4,0.0005,-3.152500,0
...,...,...,...
4999995,499.9996,2.919274,7
4999996,499.9997,2.697906,7
4999997,499.9998,4.516337,8
4999998,499.9999,5.639669,9


In [14]:
time_st = time.time()

pre_train4 = divide_and_add_features(train['signal'])
pre_test4 = divide_and_add_features(test['signal'])

pre_train4.drop(['signal'], inplace = True, axis = 1)
pre_test4.drop(['signal'], inplace = True, axis = 1)

pre_train4.reset_index(inplace = True, drop = True)
pre_test4.reset_index(inplace = True, drop = True)

pre_train4 = reduce_mem_usage(pre_train4)
pre_test4 = reduce_mem_usage(pre_test4)


# feature engineering part 2 (rolling and aggregate features)
train, test = get_batch(train, test)
pre_train1, pre_test1 = rolling_features(train, test)
pre_train1 = reduce_mem_usage(pre_train1)
pre_test1 = reduce_mem_usage(pre_test1)
pre_train2 = static_batch_features(train, 25000)
pre_train2 = reduce_mem_usage(pre_train2)
pre_test2 = static_batch_features(test, 25000)
pre_test2 = reduce_mem_usage(pre_test2)

#prob
pre_train5, pre_test5 = proba(train, test)
pre_train5 = reduce_mem_usage(pre_train5)
pre_test5 = reduce_mem_usage(pre_test5)

del train, test
gc.collect()

print(time.time()- time_st)

100%|██████████| 4/4 [00:06<00:00,  1.60s/it]


Mem. usage decreased to 133.51 Mb (75.0% reduction)
Mem. usage decreased to 53.41 Mb (75.0% reduction)


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Mem. usage decreased to 1027.84 Mb (69.8% reduction)
Mem. usage decreased to 427.14 Mb (69.0% reduction)
Mem. usage decreased to 772.48 Mb (73.0% reduction)
Mem. usage decreased to 308.99 Mb (73.0% reduction)
Mem. usage decreased to 209.81 Mb (65.6% reduction)
Mem. usage decreased to 83.92 Mb (65.6% reduction)
660.4568948745728


In [15]:
# join features for training
feat2 = [col for col in pre_train2.columns if col not in ['open_channels', 'signal', 'time', 'batch_25000', 
                                                          'batch_index_25000', 'batch_slices_25000', 'batch_slices2_25000']]
feat3 = [col for col in pre_train5.columns if col not in ['open_channels', 'signal', 'time']] #add

pre_train = pd.concat([pre_train1, pre_train2[feat2], pre_train4, pre_train5[feat3]], axis=1)
pre_test = pd.concat([pre_test1, pre_test2[feat2], pre_test4, pre_test5[feat3]], axis=1)
del pre_train1, pre_train2, pre_train4, pre_train5, pre_test1, pre_test2, pre_test4, pre_test5

In [ ]:
# # 一時保存
# pre_train.to_csv('pre_train.csv', index=False)
# pre_test.to_csv('pre_test.csv', index=False)

In [19]:
features = [col for col in pre_train.columns if col not in ['open_channels', 'time', 'batch']]
print('Training with {} features'.format(len(features)))

Training with 174 features


## Train

In [20]:
time_st = time.time()

params = {'boosting_type': 'gbdt',
          'metric': 'rmse',
          'objective': 'regression',
          'n_jobs': -1,
          'seed': 236,
          'num_leaves': 280,
          'learning_rate': 0.1, #0.026623466966581126
          'max_depth': 7, #73
          'lambda_l1': 2.959759088169741,
          'lambda_l2': 1.331172832164913,
          'bagging_fraction': 0.9655406551472153,
          'bagging_freq': 9,
          'colsample_bytree': 0.6867118652742716}

# run model and predict
round_y_pred = run_lgb(pre_train, pre_test, features, params)
submission['open_channels'] = round_y_pred
submission.to_csv('submission.csv', index = False)

print(time.time()- time_st)

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.152932	valid_1's rmse: 0.154749
[200]	training's rmse: 0.151158	valid_1's rmse: 0.154465
[300]	training's rmse: 0.149671	valid_1's rmse: 0.154256
[400]	training's rmse: 0.148301	valid_1's rmse: 0.154099
[500]	training's rmse: 0.147038	valid_1's rmse: 0.154046
[600]	training's rmse: 0.145796	valid_1's rmse: 0.153952
[700]	training's rmse: 0.144626	valid_1's rmse: 0.153936
[800]	training's rmse: 0.143505	valid_1's rmse: 0.153908
Early stopping, best iteration is:
[766]	training's rmse: 0.143876	valid_1's rmse: 0.153901
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.152881	valid_1's rmse: 0.154864
[200]	training's rmse: 0.151165	valid_1's rmse: 0.154543
[300]	training's rmse: 0.149718	valid_1's rmse: 0.154363
[400]	training's rmse: 0.148337	valid_1's rmse: 0.154232
[500]	training's rmse: 0.147022	valid_1's rmse: 0.154122
[600]	training's rmse: 0.14585	valid_1's rmse

Our oof rmse score is 0.1539231396293199
Our oof macro f1 score is 0.9390376417973191
4705.499757766724


PB:0.9413

In [ ]:
# 重要度
lgb.plot_importance(model, importance_type='split', max_num_features=30)